In [1]:
from sympy import *
from scipy import linalg
import numpy as np

In [24]:
# A = np.array([[4,5,1,0], [2,3,0,1]])
A = np.array([[1,1,1,1], [0,1,2,3]])
A = Matrix(A)
print(A)

Matrix([[1, 1, 1, 1], [0, 1, 2, 3]])


In [45]:
# Find nullspace (kernel) of A
ker = A.nullspace()

# Normalize elements of kernel to be integers
for i in range(len(ker)):
    factor = 1
    for j in ker[i]:
        if j%1 != 0:
            factor = min(factor,j%1)
    ker[i]=ker[i] / factor


In [26]:
u, v = symbols('u v')

In [29]:
# Define symbolic variables (index 0 in python but adjusted to match MAPLE)
# sym_str = 'w:' + str(A.shape[1])
sym_str = 'w1:' + str(A.shape[1]+1)
ws = symbols(sym_str)

In [46]:
def to_polynomial(coef,vars):
    res1 = 1
    res2 = 1
    for i in range(len(coef)):
        if coef[i] >= 0:
            res1 = res1*vars[i]**coef[i]
        else:
            res2 = res2*vars[i]**(-coef[i])
    res = res1 - res2
    return res

In [47]:
sys = []
for k in ker:
    sys.append(to_polynomial(k,ws))
print(sys)

[w1*w3 - w2**2, w1**2*w4 - w2**3]


In [51]:
#Computing Groebner basis
result = groebner(sys, ws[3], ws[2], ws[1], ws[0], order='lex')
list(result)

[w2**4*w4 - w2**3*w3**2,
 w1*w2**2*w4 - w2**3*w3,
 w1**2*w4 - w2**3,
 w1*w3 - w2**2]

In [52]:
#Computing Groebner basis
result = groebner(sys, ws[0], ws[1], ws[2], ws[3], order='grevlex')
list(result)

[-w1**2*w2*w4 + w1**2*w3**2, -w1**2*w4 + w1*w2*w3, -w1*w3 + w2**2]

In [53]:
#Computing Groebner basis
result = groebner(sys, ws[3], ws[2], ws[1], ws[0], order='grevlex')
list(result)

[-w1**2*w2*w4 + w1**2*w3**2, -w1**2*w4 + w1*w2*w3, -w1*w3 + w2**2]

In [50]:
#Computing Groebner basis
print(type(ws[1]))
aresult = groebner(sys, ws[0], ws[1], ws[2], ws[3], order='lex')
list(aresult)

<class 'sympy.core.symbol.Symbol'>


[w1**2*w4 - w2**3,
 w1*w2**2*w4 - w2**3*w3,
 w1*w3 - w2**2,
 w2**4*w4 - w2**3*w3**2]

In [54]:
def get_powers(polys,ws):
    for w in ws:
        list(map(lambda p: degree(p,w),polys))
    # https://docs.sympy.org/latest/tutorial/simplification.html

In [55]:
print(list(map(lambda p: degree(p,ws[2]),aresult)))

[0, 1, 1, 2]


In [15]:
print(-1 in aresult[1].args[0].args)
print(sign(aresult[1].coeff(ws[1],2)))
print((aresult[1].coeff(ws[1],4)) == -1)

True
sign(w1**2*w3**2*w4**2)
True


In [126]:
def to_matrix(polynomials, vars):
    mat = np.zeros([len(polynomials), len(vars)], dtype=int)
    for k, poly in enumerate(polynomials):
        F, opt = poly_from_expr(poly)
        terms = F.terms(order=opt.order)
        symbols = list(F.free_symbols)
        position = {}
        for j, w in enumerate(vars):
            for i, symb in enumerate(symbols):
                if symb == w:
                    position[i] = j
        for term in terms:
            powers = term[0]
            coeff = term[1]
            if coeff >= 0:
                for i, power in enumerate(powers):
                    mat[position[i],k] += power
            else:
                for i, power in enumerate(powers):
                    mat[position[i],k] -= power

    return(Matrix(mat))
    # for 
        

In [127]:
to_matrix(aresult,ws)

Matrix([
[ 2,  1,  1,  0],
[-3, -1, -2, -2],
[ 0,  1,  1,  1],
[ 1, -1,  0,  1]])